In [1]:
from __future__ import absolute_import, division, print_function

import sys
import os
import argparse
from math import log
from datetime import datetime
from tqdm import tqdm
from collections import namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical
import threading
from functools import reduce

from knowledge_graph import KnowledgeGraph
from kg_env import BatchKGEnvironment

from collections import deque
from collections import namedtuple
#from train_agent import ActorCritic
from train_DQN import DQN
############
from utils import *
#agent = DQN(env,env.state_dim,env.act_dim,gamma = args.gamma,hidden_sizes = args.hidden)

In [9]:
import torch
index = torch.tensor([20,10,25,39,5,12])  # 这些index的值，对应着point中的索引
point = torch.ones(2,40)
# 然后用index中的数对point进行各点取值
result_x = point[0,index.long()]
result_y = point[1,index] 

In [11]:
result_x

tensor([1., 1., 1., 1., 1., 1.])

In [12]:
for y in range(12,-12,-1):
    for x in range(-30,30):
        if ((x*0.05)**2+(y*0.1)**2-1)**3-(x*0.05)**2*(y*0.1)**3<=0:
            print('\n'.join(''.join('i love study'[(x-y)%10])),end='')
        else:
            print(' ',end='')
            

                ve stui l           ve stui l                            love stui love s    love stui love s                     i love stui love stui love stui love stui                  i love stui love stui love stui love stui l                i love stui love stui love stui love stui lov                love stui love stui love stui love stui love               love stui love stui love stui love stui love                ove stui love stui love stui love stui love s               ve stui love stui love stui love stui love st               e stui love stui love stui love stui love stu                stui love stui love stui love stui love stu                  ui love stui love stui love stui love stu                   i love stui love stui love stui love stui                     ove stui love stui love stui love stu                        e stui love stui love stui love stu                          stui love stui love stui love stu                             i love stui love stui lo

In [ ]:
[''.join([(
           if((x*0.05)**2+(y*0.1)**2-1)**3-(x*0.05)**2*(y*0.1)**3<=0:
               'ilovestudy'[(x-y)%10]
           else ' ')
                          for x in range(-30,30)])
                 for y in range(12,-12,-1)]

In [7]:
print('\n'.join([''.join([('ilovestudy'[(x-y)%10]if((x*0.05)**2+(y*0.1)**2-1)**3-(x*0.05)**2*(y*0.1)**3<=0 else' ')
                          for x in range(-30,30)])
                 for y in range(12,-12,-1)]))

                estudyilo           estudyilo               
            lovestudyilovestu   lovestudyilovestu           
          ilovestudyilovestudyilovestudyilovestudyi         
         ilovestudyilovestudyilovestudyilovestudyilo        
        ilovestudyilovestudyilovestudyilovestudyilove       
        lovestudyilovestudyilovestudyilovestudyiloves       
        ovestudyilovestudyilovestudyilovestudyilovest       
        vestudyilovestudyilovestudyilovestudyilovestu       
        estudyilovestudyilovestudyilovestudyilovestud       
        studyilovestudyilovestudyilovestudyilovestudy       
         udyilovestudyilovestudyilovestudyilovestudy        
          yilovestudyilovestudyilovestudyilovestudy         
          ilovestudyilovestudyilovestudyilovestudyi         
            vestudyilovestudyilovestudyilovestudy           
             studyilovestudyilovestudyilovestudy            
              udyilovestudyilovestudyilovestudy             
                ilovestu

In [2]:
def evaluate(topk_matches, test_user_products):
    """Compute metrics for predicted recommendations.
    Args:
        topk_matches: a list or dict of product ids in ascending order.
    """
    invalid_users = []
    # Compute metrics
    precisions, recalls, ndcgs, hits = [], [], [], []
    test_user_idxs = list(test_user_products.keys())
    for uid in test_user_idxs:
        if uid not in topk_matches or len(topk_matches[uid]) < 10:
            invalid_users.append(uid)
            continue
        pred_list, rel_set = topk_matches[uid][::-1], test_user_products[uid]
        if len(pred_list) == 0:
            continue

        dcg = 0.0
        hit_num = 0.0
        for i in range(len(pred_list)):
            if pred_list[i] in rel_set:
                dcg += 1. / (log(i + 2) / log(2))
                hit_num += 1
        # idcg
        idcg = 0.0
        for i in range(min(len(rel_set), len(pred_list))):
            idcg += 1. / (log(i + 2) / log(2))
            
        ndcg = dcg / idcg
        recall = hit_num / len(rel_set)
        precision = hit_num / len(pred_list)
        hit = 1.0 if hit_num > 0.0 else 0.0

        ndcgs.append(ndcg)
        recalls.append(recall)
        precisions.append(precision)
        hits.append(hit)

    avg_precision = np.mean(precisions) * 100
    avg_recall = np.mean(recalls) * 100
    avg_ndcg = np.mean(ndcgs) * 100
    avg_hit = np.mean(hits) * 100
    print('NDCG={:.3f} |  Recall={:.3f} | HR={:.3f} | Precision={:.3f} | Invalid users={}'.format(
            avg_ndcg, avg_recall, avg_hit, avg_precision, len(invalid_users)))


In [3]:
def batch_beam_search(env, model, uids, device, topk=[25, 5, 1]):
    def _batch_acts_to_masks(batch_acts):
        batch_masks = []
        for acts in batch_acts:
            num_acts = len(acts)
            act_mask = np.zeros(model.action_dim, dtype=np.uint8)
            #act_mask = np.zeros(model.act_dim, dtype=np.uint8)
            act_mask[:num_acts] = 1
            batch_masks.append(act_mask)
        return np.vstack(batch_masks)
    
    #env KGenv
    state_pool = env.reset(uids)  # numpy of [bs, dim]
    path_pool = env._batch_path  # list of list, size=bs
    probs_pool = [[] for _ in uids] #每个uid有一个列表
    model.eval()
    for hop in range(3):
        state_tensor = torch.FloatTensor(state_pool).to(device)
        acts_pool = env._batch_get_actions(path_pool, False)  # list of list, size=bs
        actmask_pool = _batch_acts_to_masks(acts_pool)  # numpy of [bs, dim]
        actmask_tensor = torch.ByteTensor(actmask_pool).to(device)
        #probs, _ = 
        model((state_tensor, actmask_tensor))  # Tensor of [bs, act_dim]
        #probs 相当于 self.Q_value
        probs = model.Q_value + actmask_tensor.float()
        #probs = probs + actmask_tensor.float()  # In order to differ from masked actions
        topk_probs, topk_idxs = torch.topk(probs, topk[hop], dim=1)  # LongTensor of [bs, k]
        topk_idxs = topk_idxs.detach().cpu().numpy()
        topk_probs = topk_probs.detach().cpu().numpy()

        new_path_pool, new_probs_pool = [], []
        for row in range(topk_idxs.shape[0]):
            path = path_pool[row]
            probs = probs_pool[row]
            for idx, p in zip(topk_idxs[row], topk_probs[row]):
                if idx >= len(acts_pool[row]):  # act idx is invalid
                    continue
                relation, next_node_id = acts_pool[row][idx]  # (relation, next_node_id)
                if relation == SELF_LOOP:
                    next_node_type = path[-1][1]
                else:
                    next_node_type = KG_RELATION[path[-1][1]][relation]
                new_path = path + [(relation, next_node_type, next_node_id)]
                new_path_pool.append(new_path)
                new_probs_pool.append(probs + [p])
        path_pool = new_path_pool
        probs_pool = new_probs_pool
        if hop < 2:
            state_pool = env._batch_get_state(path_pool)

    return path_pool, probs_pool


In [4]:
def predict_paths(policy_file, path_file, args):
    print('Predicting paths...')
    env = BatchKGEnvironment(args.dataset, args.max_acts, max_path_len=args.max_path_len, state_history=args.state_history)
    pretrain_sd = torch.load(policy_file)
    model = DQN(env,env.state_dim, env.act_dim, gamma=args.gamma, hidden_sizes=args.hidden).to(args.device)
    #agent = DQN(env,env.state_dim,env.act_dim,gamma = args.gamma,hidden_sizes = args.hidden)
    '''???'''
    model_sd = model.state_dict()
    model_sd.update(pretrain_sd)
    model.load_state_dict(model_sd)

    test_labels = load_labels(args.dataset, 'test')
    test_uids = list(test_labels.keys())

    batch_size = 16
    start_idx = 0
    all_paths, all_probs = [], []
    pbar = tqdm(total=len(test_uids)) #进度条
    while start_idx < len(test_uids):
        end_idx = min(start_idx + batch_size, len(test_uids))
        batch_uids = test_uids[start_idx:end_idx]
        paths, probs = batch_beam_search(env, model, batch_uids, args.device, topk=args.topk)
        all_paths.extend(paths)
        all_probs.extend(probs)
        start_idx = end_idx
        pbar.update(batch_size)
    predicts = {'paths': all_paths, 'probs': all_probs}
    pickle.dump(predicts, open(path_file, 'wb'))


In [5]:
def evaluate_paths(path_file, train_labels, test_labels):
    embeds = load_embed(args.dataset)
    user_embeds = embeds[USER]
    purchase_embeds = embeds[PURCHASE][0]
    product_embeds = embeds[PRODUCT]
    scores = np.dot(user_embeds + purchase_embeds, product_embeds.T)

    # 1) Get all valid paths for each user, compute path score and path probability.
    results = pickle.load(open(path_file, 'rb'))
    pred_paths = {uid: {} for uid in test_labels}
    for path, probs in zip(results['paths'], results['probs']):
        if path[-1][1] != PRODUCT:
            continue
        uid = path[0][2]
        
        if uid not in pred_paths:
            continue
        pid = path[-1][2]
        if pid not in pred_paths[uid]:
            pred_paths[uid][pid] = []
        path_score = scores[uid][pid]
        path_prob = reduce(lambda x, y: x * y, probs)
        pred_paths[uid][pid].append((path_score, path_prob, path))

    # 2) Pick best path for each user-product pair, also remove pid if it is in train set.
    best_pred_paths = {}
    for uid in pred_paths:
        train_pids = set(train_labels[uid])
        best_pred_paths[uid] = []
        for pid in pred_paths[uid]:
            if pid in train_pids:
                continue
            # Get the path with highest probability
            sorted_path = sorted(pred_paths[uid][pid], key=lambda x: x[1], reverse=True)
            best_pred_paths[uid].append(sorted_path[0])

    # 3) Compute top 10 recommended products for each user.
    sort_by = 'score'
    pred_labels = {}
    for uid in best_pred_paths:
        if sort_by == 'score':
            sorted_path = sorted(best_pred_paths[uid], key=lambda x: (x[0], x[1]), reverse=True)
        elif sort_by == 'prob':
            sorted_path = sorted(best_pred_paths[uid], key=lambda x: (x[1], x[0]), reverse=True)
        top10_pids = [p[-1][2] for _, _, p in sorted_path[:10]]  # from largest to smallest
        # add up to 10 pids if not enough
        if args.add_products and len(top10_pids) < 10:
            train_pids = set(train_labels[uid])
            cand_pids = np.argsort(scores[uid])
            for cand_pid in cand_pids[::-1]:
                if cand_pid in train_pids or cand_pid in top10_pids:
                    continue
                top10_pids.append(cand_pid)
                if len(top10_pids) >= 10:
                    break
        # end of add
        pred_labels[uid] = top10_pids[::-1]  # change order to from smallest to largest!

    evaluate(pred_labels, test_labels)


In [6]:
def test(args):
    policy_file = args.log_dir + '/dqn3_model_epoch_{}.ckpt'.format(args.epochs)
    path_file = args.log_dir + '/dqn3_paths_epoch{}.pkl'.format(args.epochs)
    '''
    policy_file = '{}/policy_model_epoch_{}.ckpt'.format(args.log_dir, epoch)
    logger.info("Save model to " + policy_file)
    torch.save(model.state_dict(), policy_file)
    '''
    train_labels = load_labels(args.dataset, 'train')
    test_labels = load_labels(args.dataset, 'test')

    if args.run_path:
        predict_paths(policy_file, path_file, args)
    if args.run_eval:
        evaluate_paths(path_file, train_labels, test_labels)

In [7]:
#if __name__ == '__main__':
boolean = lambda x: (str(x).lower() == 'true')
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default=BEAUTY, help='One of {cloth, beauty, cell, cd}')
parser.add_argument('--name', type=str, default='train_agent', help='directory name.')
parser.add_argument('--seed', type=int, default=123, help='random seed.')
parser.add_argument('--gpu', type=str, default='0', help='gpu device.')
parser.add_argument('--epochs', type=int, default=50, help='num of epochs.')
parser.add_argument('--max_acts', type=int, default=250, help='Max number of actions.')
parser.add_argument('--max_path_len', type=int, default=3, help='Max path length.')
parser.add_argument('--gamma', type=float, default=0.99, help='reward discount factor.')
parser.add_argument('--state_history', type=int, default=1, help='state history length')
parser.add_argument('--hidden', type=int, nargs='*', default=[512, 256], help='number of samples')
parser.add_argument('--add_products', type=boolean, default=False, help='Add predicted products up to 10')
parser.add_argument('--topk', type=int, nargs='*', default=[25, 5, 1], help='number of samples')
parser.add_argument('--run_path', type=boolean, default=True, help='Generate predicted path? (takes long time)')
parser.add_argument('--run_eval', type=boolean, default=True, help='Run evaluation?')
args = parser.parse_args(['--dataset',CELL])

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = TMP_DIR[args.dataset] + '/' + args.name
test(args)


Predicting paths...
Load embedding: ./tmp/Amazon_Cellphones/transe_embed.pkl


27888it [1:23:58,  6.68it/s]                                                                                           

Load embedding: ./tmp/Amazon_Cellphones/transe_embed.pkl
NDCG=0.886 |  Recall=1.452 | HR=2.330 | Precision=0.236 | Invalid users=4485


In [ ]:
class DQN(nn.Module):
    # DQN Agent
    def __init__(self, env, state_dim, act_dim, gamma=0.99,hidden_sizes=[512,256]):
        super(DQN, self).__init__()
        self.replay_buffer = deque() #双向队列 可以从左append些什么
        # init some parameters
        self.time_step = 0
        self.epsilon = INITIAL_EPSILON
        self.state_dim = 400 #state_dim
        self.action_dim = 251       
        
        self.current_net1 = nn.Linear(self.state_dim,20)
        self.current_net2 = nn.Linear(20,self.action_dim)
        
        self.target_net1 = nn.Linear(self.state_dim,20)
        self.target_net2 = nn.Linear(20,self.action_dim)
        
        '''state_dim 要改!!!'''
         #act_dim
        #env.action_space.n
        
        self.saved_actions = []        
        self.rewards = []
        self.entropy = []

    
    def forward(self,inputs):
        state,act_mask = inputs
        #act_mask = a_m
        h_1 = self.current_net1(state)
        
        h_1 = F.dropout(F.relu(h_1))
        self.Q_value = self.current_net2(h_1)
        
        h_2 = self.target_net1(state)
        h_2 = F.dropout(F.relu(h_2))
        self.target_Q_value = self.target_net2(h_2)

        #optimizer = optim.Adam(model.parameters(), lr=args.lr)
        

    def perceive(self,state,action,reward,next_state,done,optimizer):
        one_hot_action = np.zeros(self.action_dim)
        #应该要 32*action_dim
        one_hot_action[action] = 1
        self.replay_buffer.append((state,one_hot_action,reward,next_state,done))
        self.optimizer = optimizer
        if len(self.replay_buffer) > REPLAY_SIZE:
            self.replay_buffer.popleft()

        if len(self.replay_buffer) > BATCH_SIZE:
            self.train_Q_network()
    
    
    def train_Q_network(self):
        self.time_step += 1
        # Step 1: obtain random minibatch from replay memory
        minibatch = random.sample(self.replay_buffer,BATCH_SIZE)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]
        reward_batch = torch.tensor(reward_batch)
        # Step 2: calculate y
        y_batch = []
        current_Q_batch = self.Q_value
        #max_action_next = np.argmax(current_Q_batch, axis=1)
        max_action_next=torch.argmax(current_Q_batch ,dim=1)
        target_Q_batch = self.target_Q_value

        for i in range(0,BATCH_SIZE):
            done = minibatch[i][4]
            if i == 0:
                if done:
                    y_batch = torch.tensor(reward_batch[i])
                else:
                    #print('2:',len(target_Q_batch),len(target_Q_batch[0]))
                    target_Q_value = target_Q_batch[i, max_action_next[i]] #[i, i, max_action_next[i]]
                    #print('type:',type(reward_batch),type(target_Q_value))
                    #print(reward_batch)
                    #print(target_Q_value)
                    y_batch = torch.tensor(torch.tensor(reward_batch[i]) + GAMMA * target_Q_value)
            else:
                if done:
                    #y_batch = np.append(y_batch,reward_batch[i])
                    y_batch = torch.tensor(y_batch)
                    app = torch.tensor(reward_batch[i])
                    y_batch = torch.cat((y_batch,app),0)
                else :
                    #print('1:',len(target_Q_batch),len(target_Q_batch[0]))
                    #print(len(target_Q_batch[0][0]))
                    target_Q_value = target_Q_batch[i, max_action_next[i]] #[i, i,max_action_next[i]]
                    #print('type:',type(reward_batch),type(target_Q_value))
                    y_batch = torch.tensor(y_batch)
                    app = torch.tensor(reward_batch[i]+GAMMA*target_Q_value)
                    y_batch = torch.cat((y_batch,app),0)
                    #y_batch = np.append(y_batch, reward_batch[i] + GAMMA * target_Q_value)
                    
        y_batch = y_batch.reshape(32,32)
        #print(self.entropy)
        self.loss = self.entropy.mean()
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()
        
    def egreedy_action(self,batch_state,act_mask):#batch_state = state
        state = torch.FloatTensor(batch_state)#.to(device)  # Tensor [bs, state_dim]
        act_mask = torch.FloatTensor(act_mask)#.to(device)  # Tensor of [bs, act_dim]
        self((state,act_mask))
        ''''''
        act_mask = act_mask.type(torch.uint8)        
        self.Q_value[1-act_mask] = -999999.0
        self.Q_value = F.softmax(self.Q_value, dim=-1)
        
        m = Categorical(self.Q_value)#加起来应该不是1？说不准
        acts = m.sample()
        # [CAVEAT] If sampled action is out of action_space, choose the first action in action_space.
        self.entropy = torch.cat((torch.tensor(self.entropy),m.entropy()))
        #self.entropy.append(m.entropy())
        
        if random.random() <= self.epsilon:
            acts = []
            self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
            for i in range(BATCH_SIZE):
                acts.append(random.randint(0,self.action_dim - 1))
            acts_log = torch.tensor(acts)
            self.saved_actions.append(SavedAction(m.log_prob(acts_log)))#, value))
            return acts #要return一行，不能只有一个值吧!!
        else:
            self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
            acts_log = torch.tensor(acts)
            self.saved_actions.append(SavedAction(m.log_prob(acts_log)))#, value))
            return acts #np.argmax(Q_value)

        
    def action(self,batch_state,act_mask):
        batch_state = torch.tensor(batch_state,dtype=torch.float32)
        act_mask = torch.tensor(act_mask,dtype=torch.float32)
        
        self((batch_state,act_mask))
        #self.forward(batch_state,act_mask)
        ''''''
        act_mask = act_mask.type(torch.uint8)
        #self((batch_state, act_mask))        
        self.Q_value[1-act_mask] = -999999.0
        self.Q_value = F.softmax(self.Q_value, dim=-1)

        m = Categorical(self.Q_value)#加起来应该不是1？说不准 
        acts = m.sample()
        # [CAVEAT] If sampled action is out of action_space, choose the first action in action_space.
        self.entropy = torch.cat((torch.tensor(self.entropy),m.entropy()))
        #self.entropy.append(m.entropy())
        acts_log = torch.tensor(acts)
        self.saved_actions.append(SavedAction(m.log_prob(acts_log)))#, value))
        return acts